### Objective

In this notebook we would explore the `base model` that is very important from the standpoint of defining the basic constructs of the `Pydantic Data Models`. The `Pydantic` library does the following:
- Data Model: helps define the basic data attributes and their behavior, models are associated with web applications and AI models.
- Advanced features: Type Check, Serialization and de-serialization of data
- Fun fact: its used in FastAPI

In [1]:
from pydantic import BaseModel

In [2]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [3]:
p = Person(first_name="Issac", last_name="Newton", age=84)

In [4]:
p

Person(first_name='Issac', last_name='Newton', age=84)

Prior to `Pydantic` we had `Data Classes` and these were very strictly. In `Pydantic` we can still allow the instance to be defined with data types that can at least be converted to the original defined data type.

In [6]:
p = Person(first_name='100', last_name='200', age='30')

In [7]:
p

Person(first_name='100', last_name='200', age=30)

In [14]:
p = Person(first_name='100', last_name='200', age=30.0)

In the new version of `Pydantic`, we cannot:
- convert string type to int type or vice versa
- convert float type with non zero decomal type to int type.

One of the errors that encounter here is called `ValidationError`, which we could try and catch. Also by default the field are considered required of nothing is defined.

In [16]:
try:
    Person(first_name="Johny")
except Exception as ex:
    print(ex)

2 validation errors for Person
last_name
  Field required [type=missing, input_value={'first_name': 'Johny'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
age
  Field required [type=missing, input_value={'first_name': 'Johny'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [17]:
from pydantic import ValidationError

try:
    Person(first_name="Johny")
except ValidationError as ex:
    print(ex)

2 validation errors for Person
last_name
  Field required [type=missing, input_value={'first_name': 'Johny'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
age
  Field required [type=missing, input_value={'first_name': 'Johny'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [21]:
from pprint import pprint
try:
    Person(first_name="Johny")
except ValidationError as ex:
    pprint(ex.json())

('[{"type":"missing","loc":["last_name"],"msg":"Field '
 'required","input":{"first_name":"Johny"},"url":"https://errors.pydantic.dev/2.8/v/missing"},{"type":"missing","loc":["age"],"msg":"Field '
 'required","input":{"first_name":"Johny"},"url":"https://errors.pydantic.dev/2.8/v/missing"}]')


We could make one of the  attributes `optional`. There are two ways of doing this.

In [27]:
class Person(BaseModel):
    first_name: str
    last_name: str = ""
    age: int = None

In [26]:
Person(first_name="Issac", last_name="Newton")

Person(first_name='Issac', last_name='Newton', age=None)

In [28]:
Person(first_name="Issac")

Person(first_name='Issac', last_name='', age=None)

In [33]:
from typing import Optional

class Person(BaseModel):
    first_name: str
    last_name: Optional[str]
    age: Optional[int] = None

#### Serialization strategy

**Note:** Convert `Person` pydantic class to a dictionary data type or `JSON` data type.

In [34]:
p = Person(first_name="Issac", last_name="Newton")

In [35]:
p.dict()

{'first_name': 'Issac', 'last_name': 'Newton', 'age': None}

In [36]:
p.json()

'{"first_name":"Issac","last_name":"Newton","age":null}'

In [37]:
p.dict(exclude=['age'])

{'first_name': 'Issac', 'last_name': 'Newton'}

In [42]:
print(p.model_dump_json(include=['first_name', 'last_name'], indent=4))

{
    "first_name": "Issac",
    "last_name": "Newton"
}


Earlier in `pydantic<v2`, we could easily pass indent to the arguments of `json` function now we cannot do that. However, we could use the function `model_dump_json`.

When we pass `indent` in `json` function, we get the following error.

```TypeError: `dumps_kwargs` keyword arguments are no longer supported.```

#### Deserialization strategy

We could deserialize data in `Pydantic`. Here, the package simply tries to map the attributes to the closest data type.

In [44]:
from datetime import date

In [45]:
class Person(BaseModel):
    first_name: str
    last_name: str
    dob: date
    bmi: float

In [51]:
# dictionary with complex data-types
data = {
    "first_name":"Issac",
    "last_name":"Newton", 
    "dob":date(1987,1,9),
    "bmi":20.5
}

In [52]:
p = Person.parse_obj(data)

In [53]:
p

Person(first_name='Issac', last_name='Newton', dob=datetime.date(1987, 1, 9), bmi=20.5)

In [54]:
data = {
    "first_name":"Issac",
    "last_name":"Newton", 
    "dob":date(1987,1,9),
    "bmi":20
}

In [55]:
p = Person.parse_obj(data)

In [56]:
p

Person(first_name='Issac', last_name='Newton', dob=datetime.date(1987, 1, 9), bmi=20.0)

In [81]:
data = {
    "first_name":"Issac",
    "last_name":"Newton", 
    "dob":"1987-09-01", # ISO date format yyyy-MM-dd 
    "bmi":20
}

In [62]:
p = Person.parse_obj(data)

In [63]:
p

Person(first_name='Issac', last_name='Newton', dob=datetime.date(1987, 9, 1), bmi=20.0)

In [66]:
json = '''
{
    "first_name":"Issac",
    "last_name":"Newton", 
    "dob":"1987-09-01",
    "bmi":20
}
'''

In [67]:
p = Person.parse_raw(json)

In [68]:
p

Person(first_name='Issac', last_name='Newton', dob=datetime.date(1987, 9, 1), bmi=20.0)

In python, we use `snake-casing` but in JSON we use `camel-casing`. So, we use the `Field` module in `pydantic` that has `alias` attribute where we could allow this inter-operability.

In [69]:
from pydantic import Field

In [82]:
class Person(BaseModel):
    first_name: str = Field(alias='firstName')
    last_name: str = Field(alias='lastName')
    dob: date = None
    bmi: float = 0.0

In [83]:
data

{'first_name': 'Issac', 'last_name': 'Newton', 'dob': '1987-09-01', 'bmi': 20}

In [84]:
try:
    Person.parse_obj(data)
except ValidationError as ex:
    print(ex.json())

[{"type":"missing","loc":["firstName"],"msg":"Field required","input":{"first_name":"Issac","last_name":"Newton","dob":"1987-09-01","bmi":20},"url":"https://errors.pydantic.dev/2.8/v/missing"},{"type":"missing","loc":["lastName"],"msg":"Field required","input":{"first_name":"Issac","last_name":"Newton","dob":"1987-09-01","bmi":20},"url":"https://errors.pydantic.dev/2.8/v/missing"}]


In [85]:
data = {
    "firstName":"Issac",
    "lastName":"Newton", 
    "dob":"1987-09-01", # ISO date format yyyy-MM-dd 
    "bmi":20
}

In [86]:
try:
    Person.parse_obj(data)
except ValidationError as ex:
    print(ex.json())

In [87]:
p = Person.parse_obj(data)
p

Person(first_name='Issac', last_name='Newton', dob=datetime.date(1987, 9, 1), bmi=20.0)

In [88]:
p.json()

'{"first_name":"Issac","last_name":"Newton","dob":"1987-09-01","bmi":20.0}'

Surprisingly enough, when we are converting the `JSON` or dictionary to `Pydantic` object, we need to use the`alias` argument in Field submodule. But when we serialize this o